In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Data that has narratives for complaints, but only has substantiated complaints and 5 years of them


In [2]:
detailed = pd.read_excel('Oregonian Abuse records 5 years May 2016.xlsx', header=3)

We are not looking at Adult Foster Homes, so we drop them

In [3]:
detailed1 = detailed[detailed['Facility Type'] != 'AFH']

We need to join the complaints data to outcome codes that we have separately


In [4]:
codes = pd.read_excel('OLRO Outcome Codes.xlsx', header=3)

10 years of NF complaints, no narratives, broken down by facility type


Facility data

In [5]:
facilities = pd.read_csv('APD_FacilityRecords.csv')

In [6]:
NF_complaints = pd.read_excel('Copy of Oregonian Data Request Facility Abuse Records April 2016   Reviewed.xlsx',sheetname='NF Complaints')

10 years of ALF complaints, no narratives, broken down by facility type


In [7]:
ALF_complaints = pd.read_excel('Copy of Oregonian Data Request Facility Abuse Records April 2016   Reviewed.xlsx',sheetname='ALF Complaints')

10 years of RCF complaints, no narratives, broken down by facility type


In [8]:
RCF_complaints = pd.read_excel('Copy of Oregonian Data Request Facility Abuse Records April 2016   Reviewed.xlsx',sheetname='RCF Complaints')

Selecting the fields I need from the 10-year NF, RCF and ALF dataframes


In [9]:
RCF = RCF_complaints[['Abuse_Number','Facility ID','Date Received','Fac Type','Incident Date','Investigation Results','FacilityInvestResultsAbuse','FacilityInvestResultsRule','TypeOfAbuse','OutcomeCode','Severity','SanctionNumber','FineAssessed',
           'CbcAbuse']]

In [10]:
ALF = ALF_complaints[['Abuse_Number','Facility ID','Date Received','Fac Type','Incident Date','Investigation Results','FacilityInvestResultsAbuse','FacilityInvestResultsRule','TypeOfAbuse','OutcomeCode','Severity','SanctionNumber','FineAssessed',
           'CbcAbuse']]

Rename a column from one dataframe to make it consistent with the others

In [11]:
NF_complaints.rename(columns={'Abuse_CbcAbuse': 'CbcAbuse'}, inplace=True)

In [12]:
NF = NF_complaints[['Abuse_Number','Facility ID','Date Received','Fac Type','Incident Date','Investigation Results','FacilityInvestResultsAbuse','FacilityInvestResultsRule','TypeOfAbuse','OutcomeCode','Severity','SanctionNumber','FineAssessed',
           'CbcAbuse']]

Stack the three resulting dataframes on top of each other to make one big complaints dataframe ----> OUR MAIN COMPREHENSIVE COMPLAINTS DATASET

In [13]:
complaints = pd.concat([RCF,ALF,NF])

Select only the rows that have an abuse_number and are thus an actual abuse complaint


In [14]:
complaints = complaints[complaints['Abuse_Number'].notnull()]

Add two blank columns to complaints that so as to concat with detailed

In [15]:
complaints['Action Notes']=''

In [16]:
complaints['Outcome Notes']=''

Add column to identify source

In [17]:
complaints['SOURCE']='2'

Select only complaints that were substantiated

12/9/2016 --- INCLUDE LOWER-CASE SUBSTANTIATED IN FILTER OR (DUH) LOWER-CASE THE COLUMN

In [18]:
complaints = complaints.loc[(complaints['Investigation Results'] == 'Substantiated') | (complaints['FacilityInvestResultsAbuse'] == 'Substantiated')  | (complaints['FacilityInvestResultsRule'] == 'Substantiated')]

Drop the unnecessary columns in complaints

In [19]:
complaints = complaints.drop(['Investigation Results','FacilityInvestResultsAbuse','FacilityInvestResultsRule','TypeOfAbuse'], 1)

Even though 'detailed' only has substantiated complaints, still query it to only select the substantiated complaints.

In [20]:
detailed1 = detailed1.loc[(detailed1['Investigation Results'] == 'Substantiated') | (detailed1['Facility Invest Results Abuse'] == 'Substantiated')  | (detailed1['Facility Invest Results Rule'] == 'Substantiated')]

Drop the unnecessary columns in 'detailed' -- all complaints provided were substantiated

In [21]:
detailed1 = detailed1.drop(['Investigation Results', 'Facility Invest Results Abuse', 'Facility Invest Results Rule',
       'Type Of Abuse'],1)

Rename the 'detailed' and 'complaints' columns to have consistent names to be able to concatenate

In [22]:
detailed1_columns = ['ABUSE_NUMBER', 'FACID', 'DATE_RECEIVED', 'FACTYPE', 'INCIDENT_DATE',
       'OUTCOME_CODE', 'ACTION', 'OUTCOME_NOTES', 'SEVERITY',
       'SANCTION', 'FINE', 'INCIDENT_TYPE']

In [23]:
detailed1.columns = detailed1_columns

In [24]:
complaints_columns = ['ABUSE_NUMBER', 'FACID', 'DATE_RECEIVED', 'FACTYPE', 'INCIDENT_DATE',
       'OUTCOME_CODE', 'SEVERITY', 'SANCTION','FINE','INCIDENT_TYPE', 'ACTION', 'OUTCOME_NOTES','SOURCE']

In [25]:
complaints.columns = complaints_columns

In [26]:
detailed1['SOURCE']='1'

Sort columns of both tables alphabetically, for concatenation

In [27]:
complaints = complaints.reindex_axis(sorted(complaints.columns), axis=1)

In [28]:
detailed1 = detailed1.reindex_axis(sorted(detailed1.columns), axis=1)

Concatenate the two tables

In [29]:
combined = pd.concat([detailed1,complaints])

Sort the file for to have detailed first, for to de_duplicate and keep the ones we want

In [30]:
combined = combined.sort_values('SOURCE')

Fill nulls with blanks for to be able to filter by columns with nulls

In [31]:
combined = combined.fillna('')

In [32]:
combined = combined.drop_duplicates(subset='ABUSE_NUMBER',keep='first')

Select the complaints that didn't have a condition of some sort, because conditions are their own distinct thing, not necessarily tied to a complaint.

In [33]:
combined = combined[(combined['SANCTION'].str.contains('CP')) | (combined['SANCTION']=='')]

In [34]:
combined = combined[(combined['SANCTION'].str.contains('CP')) | (combined['SANCTION']=='')]

Merge combined with codes lookup so as to have the basics of what happened in each case

In [35]:
codes.columns = ['OUTCOME_CODE','OUTCOME','OUTCOME_DEFINITION']

In [36]:
combined['OUTCOME_CODE'] = combined['OUTCOME_CODE'].astype(int).astype(str)

In [37]:
codes['OUTCOME_CODE'] = codes['OUTCOME_CODE'].astype(str)

In [38]:
combined = combined.merge(codes)

In [39]:
combined.to_csv('FOR DAVE.csv')

In [40]:
combined.head()

,ABUSE_NUMBER,ACTION,DATE_RECEIVED,FACID,FACTYPE,FINE,INCIDENT_DATE,INCIDENT_TYPE,OUTCOME_CODE,OUTCOME_NOTES,SANCTION,SEVERITY,SOURCE,OUTCOME,OUTCOME_DEFINITION
0,OT105179A,The RP failed to provide appropriate care for ...,2011-01-10,385008,NF,0,2010-08-31,L,500,RV reported asking staff to change him/her pri...,,2,1,Loss of Dignity,The resident experienced embarrassment or shame.
1,RB118453,The facility failed to treat RV with dignity.,2011-12-23,5MA254,RCF,0,2011-11-11,L,500,RP2 covered Resident #1's mouth to keep her/hi...,,2,1,Loss of Dignity,The resident experienced embarrassment or shame.
2,BC153287,The facility failed to protect the Reported Vi...,2015-11-24,5MA252,RCF,,2015-10-22,L,500,Staff reported that Reported Perpetrator 2 (RP...,,2,1,Loss of Dignity,The resident experienced embarrassment or shame.
3,GP133419A,Facility failed to protect resident from inapp...,2014-02-21,5MA255,RCF,,2013-06-04,L,500,Reported Perpatrator 2 (RP2) yelled innapropri...,,2,1,Loss of Dignity,The resident experienced embarrassment or shame.
4,GP133421A,Facility failed to protect residents from inap...,2014-02-17,5MA255,RCF,,2013-06-04,L,500,Reported Perpetrator 2 yelled at Resident #1 a...,,2,1,Loss of Dignity,The resident experienced embarrassment or shame.


In another Notebook, (Facilities to Complaints QC), we establish that there are 46 FACIDs in complaints that don't correspond to facilities in the facilities table. All but one of those is in a list of closed facilities. The one that doesn't match with anything is from one solitary complaint.

Here we'll discard from the complaints all complaints that are associated with Facility IDs that aren't in in the facilities table.

In [41]:
comp_fac = combined.merge(facilities, how = 'left')[combined.merge(facilities, how = 'left')['Operator'].notnull()]

Next it makes sense to identify which facilities are Memory Care

In [42]:
comp_fac = comp_fac.fillna('')

In [43]:
comp_fac[comp_fac['Other Service'].str.contains('MC')| comp_fac['Other Service'].str.contains('ACU')]

,ABUSE_NUMBER,ACTION,DATE_RECEIVED,FACID,FACTYPE,FINE,INCIDENT_DATE,INCIDENT_TYPE,OUTCOME_CODE,OUTCOME_NOTES,...,FAC_CCMUNumber,FAC_Type,FAC_Capacity,Facility Name,Facility Address,"City, State, Zip",Other Service,County,Owner,Operator
1,RB118453,The facility failed to treat RV with dignity.,2011-12-23,5MA254,RCF,0,2011-11-11,L,500,RP2 covered Resident #1's mouth to keep her/hi...,...,5MA254,RCF,60.0,Brookdale Roseburg,3400 NW Edenbower Blvd.,"Roseburg, OR 97470","MCC, ACU 60",(541) 464-5858,Emeritus Corporation,"Brookdale Senior Living Communities, Inc."
2,BC153287,The facility failed to protect the Reported Vi...,2015-11-24,5MA252,RCF,,2015-10-22,L,500,Staff reported that Reported Perpetrator 2 (RP...,...,5MA252,RCF,24.0,Courtyard at Mt.Tabor Garden House,6323 SE Division,"Portland, OR 97206","MCC, ACU 20",(503) 788-8711,"Courtyard Assisted Members, LLC","Integral Senior Living, LLC"
5,BC146877,The facility failed to protect RV from inappro...,2014-07-14,5MA252,RCF,,2014-04-20,L,500,Resident #1 often yells when caregivers approa...,...,5MA252,RCF,24.0,Courtyard at Mt.Tabor Garden House,6323 SE Division,"Portland, OR 97206","MCC, ACU 20",(503) 788-8711,"Courtyard Assisted Members, LLC","Integral Senior Living, LLC"
6,ES118589,The facility failed to protect RV1 from verbal...,2011-12-29,5MA243,RCF,0,2011-11-26,L,500,Reported Perpetrator 2 tried to redirect Resid...,...,5MA243,RCF,79.0,Quail Park Memory Care Residences,2630 Lone Oak Way,"Eugene, OR 97404","MCC, ACU 79",(541) 607-5025,"Laurel Court of Eugene, LLC",Living Care Management
8,BH147611,Facility failed to protect resident from inapp...,2014-10-15,5MA249,RCF,,2014-06-19,l,500,RP2 was observed being verbally inappropriate ...,...,5MA249,RCF,24.0,Avamere at Sandy,17727 SE Langensand Rd,"Sandy, OR 97055","MCC, ACU 24",(503) 668-7758,"Avamere-Sandy Operations, LLC","Avamere Sandy Operations, LLC"
9,ES133072,The facility failed to protect RV from serious...,2013-05-31,5ME119,RCF,0,2013-04-24,L,500,Reported Perpetrator 2 (RP2) took a picture of...,...,5ME119,RCF,95.0,ElderHealth & Living,382-B South 58th St,"Springfield, OR 97478","MCC, ACU 95",(541) 747-5322,ElderHealth & Living Corporation,ElderHealth & Living Corporation
11,GP116507,Facility failed to protect RV from inappropria...,2011-10-28,5MA269,RCF,0,2011-03-11,L,500,RP2 was heard telling Resident #1 to shut up. ...,...,5MA269,RCF,64.0,Brookdale Grants Pass Pointe,1400 Redwood Circle,"Grants Pass, OR 97527","MCC, ACU 64",(541) 956-0190,"S-H OpCo Spring Pointe, LLC","Brookdale Senior Living Communities, Inc."
12,NB153720A,Facility failed to protect RV's from inappropr...,2016-01-18,5MA215,RCF,,2015-11-27,L,500,Reported Perpetrator 2 (RP2) and Witness 2 eng...,...,5MA215,RCF,55.0,Baycrest Memory Care,955 Kentucky Avenue,"Coos Bay, OR 97420","MCC, ACU 55",(541) 756-7715,"Bay Area Properties, LLC","Radiant Senior Living, Inc."
13,BH148459,The facility failed to treat the RV with respe...,2014-10-31,5MA217,RCF,,2014-07-17,l,500,RP2 was observed to be verbally inappropriate ...,...,5MA217,RCF,51.0,Footsteps at The Wilsonville,7600 Vlahos Drive,"Wilsonville, OR 97070","MCC, ACU 51",(503) 582-9236,"HSRE-Springs at Wilsonville Sub-TRS, LLC","The Springs Living, LLC"
14,RB105970,The facility failed to protect RV from inappro...,2011-03-11,5MA240,RCF,0,2010-12-19,L,500,RP3 has been witnessed to be rude and negative...,...,5MA240,RCF,48.0,Callahan Court Memory Care Comm.,1770 NW Valley View Drive,"Roseburg, OR 97470","MCC, ACU 48",(541) 673-3105,LSREF Golden Ops 14(OR) LLC,"Frontier Management, LLC"


In [44]:
facilities[facilities['Other Service'].str.contains('MC')| facilities['Other Service'].str.contains('ACU')]

,Facility ID,FAC_CCMUNumber,FACID,FAC_Type,FAC_Capacity,Facility Name,Facility Address,"City, State, Zip",Other Service,County,Owner,Operator
18,385104,385104,385104,NF,100.0,Hood River Care Center,729 Henderson Rd,"Hood River, OR 97031","MC, ACU 19, ECU 11",Hood River,"Care Center (Hood River) Inc.,","Prestige Care, Inc."
36,385145,385145,385145,NF,88.0,Robison Jewish Health Center,6125 SW Boundary,"Portland, OR 97221","MC 159, RCF 36, ACU 17",Multnomah,Robison Jewish Home,Robison Jewish Home
40,385150,385150,385150,NF,92.0,Molalla Manor Care Center,301 Ridings Ave,"Molalla, OR 97038","MC, ACU 14",Clackamas,Care Center (Molalla) Inc.,"Prestige Care, Inc."
50,385166,385166,385166,NF,165.0,Maryville Nursing Home,14645 SW Farmington Rd,"Beaverton, OR 97007","MC, ACU 20",Washington,Sisters of St. Mary of Oregon Maryville Corp.,Sisters of St. Mary of Oregon Maryville Corp.
58,385183,385183,385183,NF,80.0,Marquis Centennial Post Acute Rehab,725 SE 202nd Ave,"Portland, OR 97233","MC, ACU 22",Multnomah,"Marquis Companies II, Inc.","Marquis Companies I, Inc."
66,385199,385199,385199,NF,84.0,Chehalem Health & Rehab Center,1900 E Fulton St,"Newberg, OR 97132",ACU 20,Yamhill,Care Center (Chehalem) Inc.,"Prestige Care, Inc."
72,385207,385207,385207,NF,121.0,Dallas Retirement Village Health Center,377 NW Jasper Street,"Dallas, OR 97338","RCF 20,ALF 78, ACU 14",Polk,"Dallas Health Care Center, LLC",Life Care Services LLC
80,385221,385221,385221,NF,102.0,Marquis Oregon City Post Acute Rehab,1680 Molalla Avenue,"Oregon City, OR 97045","MC, ACU 20",Clackamas,"Marquis Companies II, Inc.","Marquis Companies I, Inc."
93,385240,385240,385240,NF,214.0,Marian Estates,390 Church St,"Sublimity, OR 97385","MC, ACU 51",Marion,"Ernmaur, Inc.",Marian Estates Support Services
102,385257,385257,38L938,NF,151.0,Oregon Veterans' Home,700 Veterans Drive,"The Dalles, OR 97058","MC, ACU 25",Wasco,Oregon Dept Of Veterans Affairs,Veterans Care Centers of Oregon


In [45]:
comp_fac['Other Service'].str.split(',')

0                            []
1                [MCC,  ACU 60]
2                [MCC,  ACU 20]
3                            []
4                            []
5                [MCC,  ACU 20]
6                [MCC,  ACU 79]
7                            []
8                [MCC,  ACU 24]
9                [MCC,  ACU 95]
10                           []
11               [MCC,  ACU 64]
12               [MCC,  ACU 55]
13               [MCC,  ACU 51]
14               [MCC,  ACU 48]
15                           []
16               [MCC,  ACU 48]
17               [MCC,  ACU 48]
18               [MCC,  ACU 48]
19                           []
20                           []
21                           []
22                           []
23                           []
24                           []
25                           []
26                           []
27                           []
29                           []
30                           []
                  ...          
13455   

In [46]:
facilities = facilities.fillna('')

In [47]:
facilities[facilities['Other Service']!='']

,Facility ID,FAC_CCMUNumber,FACID,FAC_Type,FAC_Capacity,Facility Name,Facility Address,"City, State, Zip",Other Service,County,Owner,Operator
1,385010,385010,385010,NF,159,Laurelhurst Village Rehabilitation Center,3060 SE Stark St,"Portland, OR 97214",ALF 89,Multnomah,"Laurelhurst Operations, LLC","Laurelhurst Operations, LLC"
2,385015,385015,385015,NF,128,Regency Gresham Nursing & Rehabilitation Center,5905 SE Powell Valley Rd,"Gresham, OR 97080","PED 12, DayCare 25",Multnomah,Regency Gresham Nursing & Rehabilitation Cente...,"Regency Pacific Management, LLC"
3,385018,385018,385018,NF,98,Providence Benedictine Nursing Center,540 South Main St,"Mt. Angel, OR 97362","ALF 52, ECU 16",Marion,Providence Health & Services - Oregon,Providence Health & Services - Oregon
6,385039,385039,385039,NF,84,Baycrest Health Center,3959 Sheridan Ave,"North Bend, OR 97459",RCF 91,Coos,"Bay Area Properties, LLC","Radiant Senior Living, Inc."
8,385045,385045,385045,NF,99,Porthaven Healthcare Center,5330 NE Prescott,"Portland, OR 97218",DayCare 3,Multnomah,Care Center (Porthaven) Inc.,"Prestige Care, Inc."
18,385104,385104,385104,NF,100,Hood River Care Center,729 Henderson Rd,"Hood River, OR 97031","MC, ACU 19, ECU 11",Hood River,"Care Center (Hood River) Inc.,","Prestige Care, Inc."
26,385126,385126,385126,NF,117,Avamere at Three Fountains,835 Crater Lake Ave,"Medford, OR 97504","RCF 20, ALF 85",Jackson,"Waterford Operations, LLC","Waterford Operations, LLC"
36,385145,385145,385145,NF,88,Robison Jewish Health Center,6125 SW Boundary,"Portland, OR 97221","MC 159, RCF 36, ACU 17",Multnomah,Robison Jewish Home,Robison Jewish Home
40,385150,385150,385150,NF,92,Molalla Manor Care Center,301 Ridings Ave,"Molalla, OR 97038","MC, ACU 14",Clackamas,Care Center (Molalla) Inc.,"Prestige Care, Inc."
43,385155,385155,385155,NF,114,Forest Grove Rehabilitation and Care Center,3900 Pacific Ave,"Forest Grove, OR 97116",ECU 10,Washington,"PCI Care Venture I, Inc.","Prestige Care, Inc."


In [48]:
facilities['Other Service'][facilities['Other Service'].str.contains('ACU')].str.extract('(ACU \d+)').count()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


179

In [49]:
facilities['Other Service'][facilities['Other Service'].str.contains('MC \d+')]

36    MC 159, RCF 36, ACU 17
Name: Other Service, dtype: object

In [57]:
facilities[facilities['Other Service']=='MC 159, RCF 36, ACU 17']

,Facility ID,FAC_CCMUNumber,FACID,FAC_Type,FAC_Capacity,Facility Name,Facility Address,"City, State, Zip",Other Service,County,Owner,Operator,mc_capacity
36,385145,385145,385145,NF,88,Robison Jewish Health Center,6125 SW Boundary,"Portland, OR 97221","MC 159, RCF 36, ACU 17",Multnomah,Robison Jewish Home,Robison Jewish Home,17


In [50]:
facilities['Other Service'][facilities['Other Service'].str.contains('ACU')].str.extract('(ACU \d+)').str.extract('(\d+)')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


18      19
36      17
40      14
50      20
58      22
66      20
72      14
80      20
93      51
102     25
119     20
132     17
134     42
137     44
139     60
140     48
141     22
142     58
143     24
144     59
147     20
154     32
155     26
158     50
167     30
170     60
177     28
178     64
190     16
194     44
      ... 
385     48
386     30
388     57
389    114
390     32
391     50
393     37
394     60
396     56
397     42
398     55
399     51
400     42
401     55
402     60
405     48
407     56
408     79
409     24
411     24
412     20
413     60
415     22
416     24
417     30
418     64
419     95
487     23
638     23
642     24
Name: Other Service, dtype: object

In [51]:
facilities['Other Service'].str.extract('(ACU \d+)').str.extract('(\d+)')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18      19
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
      ... 
613    NaN
614    NaN
615    NaN
616    NaN
617    NaN
618    NaN
619    NaN
620    NaN
621    NaN
622    NaN
623    NaN
624    NaN
625    NaN
626    NaN
627    NaN
628    NaN
629    NaN
630    NaN
631    NaN
632    NaN
633    NaN
634    NaN
635    NaN
636    NaN
637    NaN
638     23
639    NaN
640    NaN
641    NaN
642     24
Name: Other Service, dtype: object

In [52]:
facilities['Other Service'][facilities['Other Service'].str.contains('ACU') & facilities['Other Service'].str.contains('MC')]

18         MC, ACU 19, ECU 11
36     MC 159, RCF 36, ACU 17
40                 MC, ACU 14
50                 MC, ACU 20
58                 MC, ACU 22
80                 MC, ACU 20
93                 MC, ACU 51
102                MC, ACU 25
132                MC, ACU 17
137               MCC, ACU 44
139               MCC, ACU 60
140               MCC, ACU 48
141               MCC, ACU 22
142               MCC, ACU 58
143               MCC, ACU 24
144               MCC, ACU 59
147               MCC, ACU 20
154               MCC, ACU 32
155               MCC, ACU 26
158               MCC, ACU 50
167               MCC, ACU 30
170               MCC, ACU 60
177               MCC, ACU 28
178               MCC, ACU 64
190               MCC, ACU 16
194               MCC, ACU 44
195               MCC, ACU 16
197               MCC, ACU 30
198        MCC, ACU 22, ECU 6
199               MCC, ACU 35
                ...          
385               MCC, ACU 48
386               MCC, ACU 30
388       

In [53]:
facilities['mc_capacity']=facilities['Other Service'].str.extract('(ACU \d+)').str.extract('(\d+)')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [54]:
facilities[facilities['mc_capacity'].notnull()]

,Facility ID,FAC_CCMUNumber,FACID,FAC_Type,FAC_Capacity,Facility Name,Facility Address,"City, State, Zip",Other Service,County,Owner,Operator,mc_capacity
18,385104,385104,385104,NF,100,Hood River Care Center,729 Henderson Rd,"Hood River, OR 97031","MC, ACU 19, ECU 11",Hood River,"Care Center (Hood River) Inc.,","Prestige Care, Inc.",19
36,385145,385145,385145,NF,88,Robison Jewish Health Center,6125 SW Boundary,"Portland, OR 97221","MC 159, RCF 36, ACU 17",Multnomah,Robison Jewish Home,Robison Jewish Home,17
40,385150,385150,385150,NF,92,Molalla Manor Care Center,301 Ridings Ave,"Molalla, OR 97038","MC, ACU 14",Clackamas,Care Center (Molalla) Inc.,"Prestige Care, Inc.",14
50,385166,385166,385166,NF,165,Maryville Nursing Home,14645 SW Farmington Rd,"Beaverton, OR 97007","MC, ACU 20",Washington,Sisters of St. Mary of Oregon Maryville Corp.,Sisters of St. Mary of Oregon Maryville Corp.,20
58,385183,385183,385183,NF,80,Marquis Centennial Post Acute Rehab,725 SE 202nd Ave,"Portland, OR 97233","MC, ACU 22",Multnomah,"Marquis Companies II, Inc.","Marquis Companies I, Inc.",22
66,385199,385199,385199,NF,84,Chehalem Health & Rehab Center,1900 E Fulton St,"Newberg, OR 97132",ACU 20,Yamhill,Care Center (Chehalem) Inc.,"Prestige Care, Inc.",20
72,385207,385207,385207,NF,121,Dallas Retirement Village Health Center,377 NW Jasper Street,"Dallas, OR 97338","RCF 20,ALF 78, ACU 14",Polk,"Dallas Health Care Center, LLC",Life Care Services LLC,14
80,385221,385221,385221,NF,102,Marquis Oregon City Post Acute Rehab,1680 Molalla Avenue,"Oregon City, OR 97045","MC, ACU 20",Clackamas,"Marquis Companies II, Inc.","Marquis Companies I, Inc.",20
93,385240,385240,385240,NF,214,Marian Estates,390 Church St,"Sublimity, OR 97385","MC, ACU 51",Marion,"Ernmaur, Inc.",Marian Estates Support Services,51
102,385257,385257,38L938,NF,151,Oregon Veterans' Home,700 Veterans Drive,"The Dalles, OR 97058","MC, ACU 25",Wasco,Oregon Dept Of Veterans Affairs,Veterans Care Centers of Oregon,25


In [56]:
#facilities.to_csv('for second notebook cause too much.csv')